# Projeto de Classificação para Marinha do Brasil

## Autor: Vinícius dos Santos Mello (viniciusdsmello@poli.ufrj.br)

Laboratório de Processamento de Sinais - UFRJ

Laboratório de Tecnologia Sonar

Instituto de Pesquisas da Marinha - IPqM

In [1]:
import os
import pickle
import numpy as np
import time

from keras.utils import np_utils
from keras.models import load_model

import sklearn.metrics
from sklearn.externals import joblib

import matplotlib.pyploy as plt

from Functions import TrainParameters as trnparams
from Functions import TrainFunctions

import multiprocessing 

init_time = time.time()

m_time = time.time()
print 'Time to import all libraries: '+str(m_time-init_time)+' seconds'

analysis_name = 'StackedAutoEncoder'

# Enviroment variables
data_path = os.getenv('OUTPUTDATAPATH')
results_path = os.getenv('PACKAGE_NAME')

# paths to export results
base_results_path = '%s/%s'%(results_path,analysis_name)
pict_results_path = '%s/pictures_files'%(base_results_path)
files_results_path = '%s/output_files'%(base_results_path)

# For multiprocessing purpose
num_processes = multiprocessing.cpu_count()

# Read data
m_time = time.time()

# Database caracteristics
database = '4classes'
n_pts_fft = 1024
decimation_rate = 3
spectrum_bins_left = 400
development_flag = False
development_events = 400

# Check if LofarData has created...
if not os.path.exists('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                      (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left)):
    print 'No Files in %s/%s\n'%(data_path,database)
else:
    #Read lofar data
    [data,trgt,class_labels] = joblib.load('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                                           (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left))


    m_time = time.time()-m_time
    print 'Time to read data file: '+str(m_time)+' seconds'

    # correct format
    all_data = data
    all_trgt = trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))
    
    # Process data
    # unbalanced data to balanced data with random data creation of small classes

    # Same number of events in each class
    qtd_events_biggest_class = 0
    biggest_class_label = ''

    for iclass, class_label in enumerate(class_labels):
        if sum(all_trgt==iclass) > qtd_events_biggest_class:
            qtd_events_biggest_class = sum(all_trgt==iclass)
            biggest_class_label = class_label
        print "Qtd event of %s is %i"%(class_label,sum(all_trgt==iclass))
    print "\nBiggest class is %s with %i events"%(biggest_class_label,qtd_events_biggest_class)

    balanced_data = {}
    balanced_trgt = {}

    from Functions import DataHandler as dh
    m_datahandler = dh.DataHandlerFunctions()

    for iclass, class_label in enumerate(class_labels):
        if development_flag:
            class_events = all_data[all_trgt==iclass,:]
            if len(balanced_data) == 0:
                balanced_data = class_events[0:development_events,:]
                balanced_trgt = (iclass)*np.ones(development_events)
            else:
                balanced_data = np.append(balanced_data,
                                          class_events[0:development_events,:], 
                                          axis=0)
                balanced_trgt = np.append(balanced_trgt,(iclass)*np.ones(development_events))
        else:
            if len(balanced_data) == 0:
                class_events = all_data[all_trgt==iclass,:]
                balanced_data = m_datahandler.CreateEventsForClass(
                    class_events,qtd_events_biggest_class-(len(class_events)))
                balanced_trgt = (iclass)*np.ones(qtd_events_biggest_class)
            else:
                class_events = all_data[all_trgt==iclass,:]
                created_events = (m_datahandler.CreateEventsForClass(all_data[all_trgt==iclass,:],
                                                                     qtd_events_biggest_class-
                                                                     (len(class_events))))
                balanced_data = np.append(balanced_data,created_events,axis=0)
                balanced_trgt = np.append(balanced_trgt,
                                          (iclass)*np.ones(created_events.shape[0]),axis=0)
        
    all_data = balanced_data
    all_trgt = balanced_trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))

Using Theano backend.
/home/vinicius.mello/.virtualenvs/sonarenvAnnecy/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Time to import all libraries: 2.40802764893e-05 seconds
Time to read data file: 1.02640604973 seconds
Qtd event of 0 is 12939
Qtd event of 1 is 29352
Qtd event of 2 is 11510
Qtd event of 3 is 23760

Biggest class is 1 with 29352 events
DataHandler Class: CreateEventsForClass
Original Size: (12939, 400)
DataHandler Class: CreateEventsForClass
Original Size: (29352, 400)
DataHandler Class: CreateEventsForClass
Original Size: (11510, 400)
DataHandler Class: CreateEventsForClass
Original Size: (23760, 400)


In [2]:
%time 
# Load train parameters

analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'

trn_params_folder='%s/%s/%s_trnparams.jbl'%(results_path,analysis_str,analysis_name)
#os.remove(trn_params_folder)
if not os.path.exists(trn_params_folder):
    trn_params = trnparams.SAENoveltyDetectionTrnParams(n_inits=2,
                                                       hidden_activation='tanh', # others tanh, relu, sigmoid, linear 
                                                       output_activation='linear',
                                                       n_epochs=500,
                                                       patience=30,
                                                       batch_size=256,
                                                       verbose=False)
    trn_params.save(trn_params_folder)
else:
    trn_params = trnparams.SAENoveltyDetectionTrnParams()
    trn_params.load(trn_params_folder)
    
# Choose how many fold to be used in Cross Validation
n_folds = 2
CVO = trnparams.NoveltyDetectionFolds(folder=results_path,n_folds=n_folds,trgt=all_trgt,dev=development_flag)
print trn_params.get_params_str()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs
2_inits_mapstd_norm_500_epochs_256_batch_size_tanh_hidden_activation_linear_output_activation


# Treinamento com Processamento Pararalelo

In [3]:
# Train example

# Choose neurons topology
max_n_neurons = 475
min_n_neurons = 0
neurons_step = 75

verbose = False

# Create neurons vector to be used in multiprocessing.Pool()
neurons_mat = [1,100,200,250,300,350,400,450]  #range(min_n_neurons, max_n_neurons, neurons_step)
print trn_params.get_params_str()
start_time = time.time()
for inovelty, novelty_class in enumerate(np.unique(trgt)):
    trn_data = all_data[all_trgt!=novelty_class]
    trn_trgt = all_trgt[all_trgt!=novelty_class]

    trn_trgt[trn_trgt>novelty_class] = trn_trgt[trn_trgt>novelty_class]-1
    
    if inovelty != 0:
        print ''
    print 'Novelty class: %i'%inovelty
    
    def trainNeuron(ineuron):
        n_folds = len(CVO[inovelty])
        for ifold in range(n_folds):
            #print 'Neuron value: %i - fold %i'%(ineuron, ifold)
            TrainFunctions.SAENoveltyTrainFunction(data=trn_data,
                                                       trgt=trn_data,
                                                       inovelty=inovelty,
                                                       ifold=ifold,
                                                       n_folds=n_folds,
                                                       n_neurons=ineuron,
                                                       trn_params=trn_params,
                                                       save_path=results_path,
                                                       layer = 1,
                                                       verbose=verbose,
                                                       dev=development_flag)
    
    
    # Start Parallel processing
    p = multiprocessing.Pool(processes=num_processes)
    
    # To train on multiple cores sweeping the number of neurons
    results = p.map(trainNeuron, neurons_mat)
            
    p.close()
    p.join()        

end_time = time.time() - start_time
print "It took %.3f seconds to perform the training"%(end_time)

2_inits_mapstd_norm_500_epochs_256_batch_size_tanh_hidden_activation_linear_output_activation
Novelty class: 0

Novelty class: 1

Novelty class: 2
Neuron: 250 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 100 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 1 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 400 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 300 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 200 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 350 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 450 - Fold 1 of 2 Folds -  Init 1 of 2 Inits
Neuron: 1 - Fold 1 of 2 Folds -  Init 2 of 2 Inits
Neuron: 1 - Fold 2 of 2 Folds -  Init 1 of 2 Inits
Neuron: 1 - Fold 2 of 2 Folds -  Init 2 of 2 Inits
Neuron: 100 - Fold 1 of 2 Folds -  Init 2 of 2 Inits
Neuron: 400 - Fold 1 of 2 Folds -  Init 2 of 2 Inits
Neuron: 200 - Fold 1 of 2 Folds -  Init 2 of 2 Inits
Neuron: 100 - Fold 2 of 2 Folds -  Init 1 of 2 Inits
Neuron: 250 - Fold 1 of 2 Folds -  Init 2 of 2 Inits
Neuron: 450 -

In [4]:
# Reconstruction of Known Classes vs Reconstruction of Novelty - novelty detection for neural network
# threshold sweep

# generate analysis data
save_path=results_path

analysis_str = 'StackedAutoEncoders'
model_prefix_str = 'RawData'

analysis_file_name='%s/%s/%s_novelty_detection_thr_sweep.jbl'%(results_path,analysis_str,analysis_name)

#os.remove(analysis_file_name)

ineuron = 400

thr_mat = np.arange(0.0,1.05,0.05)
"""
# Check if the analysis has already been performed
if not os.path.exists(analysis_file_name):
    trn_params_folder='%s/%s/%s_trnparams.jbl'%(results_path,analysis_str,analysis_name)
    
    if not os.path.exists(trn_params_folder):
        trn_params = trnparams.NNNoveltyDetectionTrnParams(n_inits=2,
                                                           hidden_activation='tanh',
                                                           output_activation='linear',
                                                           n_epochs=500,
                                                           patience=30,
                                                           batch_size=256,
                                                           verbose=False)
        trn_params.save(trn_params_folder)
    else:
        trn_params = trnparams.NNNoveltyDetectionTrnParams()
        trn_params.load(trn_params_folder)

    params_str = trn_params.get_params_str()
    n_folds = 2
    CVO = trnparams.NoveltyDetectionFolds(folder=results_path,n_folds=n_folds,trgt=all_trgt,dev=development_flag)
    
    n_classes = len(np.unique(all_trgt))
    len_thr_mat = len(thr_mat)
    
    class_error_mat = np.zeros([n_folds,n_classes,n_classes,len_thr_mat])
    novelty_error_mat = np.zeros([n_folds,n_classes,len_thr_mat])
    
    for inovelty, novelty_class in enumerate(np.unique(trgt)):
        trn_data = all_data[all_trgt!=novelty_class]
        trn_trgt = all_trgt[all_trgt!=novelty_class]
        
        trn_trgt[trn_trgt>novelty_class] = trn_trgt[trn_trgt>novelty_class]-1
        
        for ifold in range(len(CVO[inovelty])):
            print 'Novelty class: %01.0f - neuron: %i - fold %i'%(novelty_class, ineuron, ifold)
            [classifier,trn_desc] = TrainFunctions.SAENoveltyTrainFunction(data=trn_data,
                                                                          trgt=trn_trgt,
                                                                          inovelty=inovelty,
                                                                          ifold=ifold,
                                                                          n_folds=len(CVO[inovelty]),
                                                                          n_neurons=ineuron, 
                                                                          trn_params=trn_params,
                                                                          save_path=results_path,
                                                                          dev=development_flag)
            output = classifier.predict(trn_data)
            novelty_output = classifier.predict(all_data[all_trgt==novelty_class])
            
            for ithr,thr_value in enumerate(thr_mat): 
                buff = np.zeros([len(np.unique(trgt))-1])
                for iclass, class_id in enumerate(np.unique(trgt)):
                    if iclass == inovelty:
                        continue
                    class_eff_mat[ifold, inovelty, iclass, ithr] = float(sum(output[np.argmax(output,axis=1)==iclass-(iclass>inovelty),iclass-(iclass>inovelty)]>thr_value))/float(len(output))
                    buff[iclass-(iclass>inovelty)] = float(sum(output[np.argmax(output,axis=1)==iclass-(iclass>inovelty),iclass-(iclass>inovelty)]>thr_value))/float(len(output))
                novelty_eff_mat[ifold, inovelty, ithr] = float(sum(1-(novelty_output>thr_value).any(axis=1)))/float(len(novelty_output))
                known_acc_mat[ifold, inovelty, ithr] = np.mean(buff,axis=0)
                known_sp_mat[ifold, inovelty, ithr]= (np.sqrt(np.mean(buff,axis=0)
                                                              *np.power(np.prod(buff),1./float(len(buff)))))
    joblib.dump([class_error_mat, novelty_error_mat, thr_mat],
                analysis_file_name,compress=9)
else:
    [class_eff_mat, novelty_eff_mat, known_acc_mat, known_sp_mat, thr_mat] = joblib.load(analysis_file_name) 

# plot analysis
import matplotlib.pyplot as plt
%matplotlib inline  

fig = plt.subplots(figsize=(20,15))

plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.numpoints'] = 1
plt.rc('legend',**{'fontsize':15})
plt.rc('font', weight='bold')

m_colors = ['b', 'r', 'g', 'y']

for inovelty, novelty_class in enumerate(np.unique(all_trgt)):
    ax = plt.subplot(2,2,inovelty+1)
    for iclass, m_class in enumerate(np.unique(all_trgt)):
        if novelty_class == m_class:
            #a = 0
            ax.errorbar(thr_mat,np.mean(novelty_eff_mat[:,int(novelty_class),:],axis=0),
                        np.std(novelty_eff_mat[:,int(novelty_class),:],axis=0),fmt='o-',
                        color='k',alpha=0.7,linewidth=2.5,
                        label='Novelty. Det.')
            ax.errorbar(thr_mat,np.mean(known_acc_mat[:,int(novelty_class),:],axis=0),
                        np.std(known_acc_mat[:,int(novelty_class),:],axis=0),fmt='o--',
                        color='k',alpha=0.7,linewidth=2.5,
                        label='Known Acc.')
            ax.errorbar(thr_mat,np.mean(known_sp_mat[:,int(novelty_class),:],axis=0),
                        np.std(known_sp_mat[:,int(novelty_class),:],axis=0),fmt='o:',
                        color='k',alpha=0.7,linewidth=2.5,
                        label='Known SP')
        else:
            ax.errorbar(thr_mat,np.mean(class_eff_mat[:,int(novelty_class),int(m_class),:],axis=0),
                        np.std(class_eff_mat[:,int(novelty_class),int(m_class),:],axis=0),fmt='o-',
                        color=m_colors[int(m_class)],alpha=0.7,linewidth=2.5,
                       label='C%i Eff.'%(int(m_class)+1))
    ax.set_xticks(thr_mat)
    ax.set_xticklabels(thr_mat,rotation=45)
    ax.set_title('Eff per Known Class',fontsize=18,weight='bold')
    ax.set_xlim([np.min(thr_mat), np.max(thr_mat)])
    ax.set_ylim([-0.5,1.2])
    ax.grid()
    
    if inovelty > 1:
        ax.set_xlabel('Threshold',fontsize=18,weight='bold')
    if inovelty == 0 or inovelty == 2:
        ax.set_ylabel('Eff.',fontsize=18,weight='bold')
        
    handles, labels = ax.get_legend_handles_labels()
    # sort both labels and handles by labels
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
    ax.legend(handles, labels, ncol=1)"""

"\n# Check if the analysis has already been performed\nif not os.path.exists(analysis_file_name):\n    trn_params_folder='%s/%s/%s_trnparams.jbl'%(results_path,analysis_str,analysis_name)\n    \n    if not os.path.exists(trn_params_folder):\n        trn_params = trnparams.NNNoveltyDetectionTrnParams(n_inits=2,\n                                                           hidden_activation='tanh',\n                                                           output_activation='linear',\n                                                           n_epochs=500,\n                                                           patience=30,\n                                                           batch_size=256,\n                                                           verbose=False)\n        trn_params.save(trn_params_folder)\n    else:\n        trn_params = trnparams.NNNoveltyDetectionTrnParams()\n        trn_params.load(trn_params_folder)\n\n    params_str = trn_params.get_params_str()\n    n_